In [ ]:
!pip install nltk

In [ ]:
!pip install wordcloud

In [ ]:
import nltk

In [ ]:
nltk.download()

In [ ]:
import pandas as pd,numpy as np
import matplotlib.pyplot as plt

In [ ]:
pwd

In [ ]:
reviews = pd.read_csv("K8 Reviews.csv")

In [ ]:
reviews.head(10)

This dataset is scrapped from amazon about Lenevo K8 mobile phones.While the users gave the reviews, they also give the ratings
- Ratings 1,2,3 -> negitive sentiment has a value 0 in the sentiment column
- Ratings 4,5 -> positive sentiment has a value 1 in the sentiment column

In [ ]:
reviews.sentiment.value_counts(normalize = True)

In [ ]:
reviews.shape

In [ ]:
from wordcloud import WordCloud

In [ ]:
reviews_combined = " ".join(reviews.review.values)

In [ ]:
reviews_combined

In [ ]:
#Total words in reviews
len(reviews_combined)

In [ ]:
reviews_combined[:2000]

In [ ]:
word_cloud = WordCloud().generate_from_text(reviews_combined)

In [ ]:
?WordCloud

In [ ]:
plt.figure()
plt.imshow(word_cloud)
plt.show()

In [ ]:
word_cloud = WordCloud(width=800,height=800,background_color='white',max_words=50,random_state = 46333, stopwords = None).\
generate_from_text(reviews_combined)

In [ ]:
plt.figure(figsize=[8,8])
plt.imshow(word_cloud)
plt.show()

In [ ]:
all_terms = []
fdist = {}
all_terms = reviews_combined.split(" ")
for word in all_terms:
    fdist[word] = fdist.get(word,0) + 1

In [ ]:
fdist["."]

In [ ]:
fdist["will"]

In [ ]:
fdist.get("good",4646464560)

In [ ]:
fdist.get("goodjghjfgjfjh",4646464560)

In [ ]:
print(all_terms)

In [ ]:
len(all_terms)

In [ ]:
len(set(all_terms))

In [ ]:
freq = {"words":list(fdist.keys()),"freq":list(fdist.values())}
df_dist = pd.DataFrame(freq)

In [ ]:
df_dist

In [ ]:
%matplotlib inline
df_dist.sort_values(ascending=False, by="freq").head(25).\
plot.bar(x= "words", y= "freq",figsize=(20,5)) 

In [ ]:
fdist["good."]+fdist["Good"]+fdist["good,"]

In [ ]:
print(fdist["good"],fdist["Good"])

Problems with the above visuals
    1.case is non uniform
    2. punctuations and stop words
    3. words with different tense forms like charged, charging, charges has its root form in charge

In [ ]:
#1. case normalization and tokenizing

In [ ]:
reviews.review[:10]

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
#All Reviews tokenized and in lower case
all_terms = word_tokenize(reviews_combined.lower())

In [ ]:
print(all_terms)

In [ ]:
print(all_terms[:20])

In [ ]:
len(all_terms)

In [ ]:
len(set(all_terms))

In [ ]:
#visualizing the frequency distribution

In [ ]:
from nltk.probability import FreqDist

In [ ]:
fdist = FreqDist(all_terms)
fdist

In [ ]:
fdist.plot(30,cumulative=False)
plt.show()

In [ ]:
from string import punctuation
from nltk.corpus import stopwords

In [ ]:
stop_nltk = stopwords.words("english")

In [ ]:
stop_updated = stop_nltk + list(punctuation) + ["n't","....","....."]+["'s","..","...","amazon","phone", "mobile","lenovo","k8","product"]

In [ ]:
print(stop_updated)

In [ ]:
stop_updated.remove('not')

In [ ]:
#stop_updated.remove("n't")
print(stop_updated)

below code removes:
1. stopwords, 
2. punct's,
3. custom stop words,
4. words with lenght less than 2

In [ ]:
terms_updated = [word for word in all_terms if word not in stop_updated and len(word) > 1]

In [ ]:
print(terms_updated[:20])

In [ ]:
len(set(terms_updated))

In [ ]:
len(terms_updated)

In [ ]:
fdist = FreqDist(terms_updated)
fdist

In [ ]:
fdist.plot(30,cumulative=False)
plt.show()

In [ ]:
cleanrevstring = " ".join(terms_updated)

In [ ]:
word_cloud = WordCloud(width=800,height=800,background_color='white',max_words=50,random_state = 46333, stopwords = None,collocations = False).\
generate_from_text(cleanrevstring)

In [ ]:
plt.figure(figsize=[8,8])
plt.imshow(word_cloud)
plt.show()

In [ ]:
word_cloud = WordCloud(width=800,height=800,background_color='white',max_words=50,random_state = 46333, stopwords = None,collocations = True).\
generate_from_text(cleanrevstring)

In [ ]:
plt.figure(figsize=[8,8])
plt.imshow(word_cloud)
plt.show()

# 

In [ ]:
from nltk.stem import SnowballStemmer
stemmer_s = SnowballStemmer("english")

In [ ]:
reviews_updated_stem = [stemmer_s.stem(word) for word in terms_updated]

In [ ]:
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()

In [ ]:
reviews_updated_lemm = [lemm.lemmatize(word,pos='v') for word in terms_updated]

In [ ]:
print(len(set(reviews_updated_stem)),len(set(reviews_updated_lemm)))
print(len((reviews_updated_stem)),len((reviews_updated_lemm)))

In [ ]:
#Stememr reduces the number of distinct words in the corpus to a greater extent

In [ ]:
# Define function that takes in a raw string, returns a clean string
# - Tokenize, lower case, remove stop words, stem, stitch back to string
def clean_txtstem(sent):
    tokens = word_tokenize(sent.lower())
    stemmed = [stemmer_s.stem(term) for term in tokens if term not in stop_updated and len(term) > 1] 
    res = " ".join(stemmed)
    return res
# - Tokenize, lower case, remove stop words, lemma, stitch back to string
def clean_txtlemm(sent):
    tokens = word_tokenize(sent.lower())
    stemmed = [lemm.lemmatize(term, pos = 'v')  for term in tokens if term not in stop_updated and len(term) > 1] 
    res = " ".join(stemmed)
    return res

In [ ]:
%%time
reviews['clean_reviewstem'] = reviews.review.apply(clean_txtstem)

In [ ]:
%%time
reviews['clean_reviewlemm'] = reviews.review.apply(clean_txtlemm)

In [ ]:
#reviews = reviews.drop('clean_reviewlemm1', axis = 1)

In [ ]:
reviews.head()

# Word cloud on cleaned dataset

In [ ]:
reviews_combinedstem = " ".join(reviews.clean_reviewstem.values)
reviews_combinedlemm = " ".join(reviews.clean_reviewlemm.values)

In [ ]:
word_cloud = WordCloud(width=800,height=800,background_color='white',max_words=150).\
generate_from_text(reviews_combinedstem)

In [ ]:
plt.figure(figsize=[8,8])
plt.imshow(word_cloud)
plt.show()

In [ ]:
word_cloud = WordCloud(width=800,height=800,background_color='white',max_words=50, random_state = 59,stopwords=[]).\
generate_from_text(reviews_combinedlemm)
word_cloud = WordCloud(width=800,height=800,background_color='white',max_words=150, random_state = 59).\
generate_from_text(reviews_combinedlemm)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
plt.figure(figsize=[10,10])
plt.imshow(word_cloud)
plt.show()

In [ ]:
from PIL import Image
mask = np.array(Image.open('download.jpg'))
wc = WordCloud(mask=mask, background_color="white",
               max_words=100, max_font_size=256,
               random_state=42, width=mask.shape[1],
               height=mask.shape[0])
wc.generate(reviews_combinedlemm)
#plt.imshow(wc, interpolation="bilinear")
plt.figure(figsize=[10,10])
plt.imshow(wc, interpolation="bilinear")
plt.axis('off')
plt.show()

# Bar Graph on top 25 words

In [ ]:
all_terms = []
fdist = {}
all_terms = reviews_combinedstem.split(" ")
for word in all_terms:
    fdist[word] = fdist.get(word,0) + 1

In [ ]:
all_terms = []
fdist = {}
all_terms = reviews_combinedlemm.split(" ")
for word in all_terms:
    fdist[word] = fdist.get(word,0) + 1

In [ ]:
freq = {"words":list(fdist.keys()),"freq":list(fdist.values())}
df_dist = pd.DataFrame(freq)

In [ ]:
%matplotlib inline
df_dist.sort_values(ascending=False, by="freq").head(25).\
plot.bar(x= "words", y= "freq",figsize=(20,5)) 

# Plot the bar graph for top 25 frequenctly used bigrams

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# create a bigram count vectorizer object
#bigram_count_vectorizer = CountVectorizer(ngram_range=(2,2))
bigram_count_vectorizer = CountVectorizer(ngram_range=(2,2),max_features = 1000)

In [ ]:
X_bigram = bigram_count_vectorizer.fit_transform(reviews['clean_reviewlemm'])

# Creating a DTM
DTM_bigram = pd.DataFrame(X_bigram.toarray(), columns=bigram_count_vectorizer.get_feature_names())

In [ ]:
DTM_bigram

In [ ]:
bigram_count_vectorizer.get_feature_names()

In [ ]:
X_bigram

In [ ]:
X_bigram.toarray()

In [ ]:
X_bigram.todense()

In [ ]:
bigram_count_vectorizer.get_feature_names()[:200]

In [ ]:
DTM_bigram.head(10)

In [ ]:
DTM_bigram.sum().sort_values(ascending=False).head(50).plot.bar(figsize=(20,5)) 